INSTALL
- FAISS - 
- Sent2Vec

In [ ]:
import numpy as np
import faiss   

In [ ]:
import os
import time
name = "OPUS-bert-"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd


os.chdir(path)


# import os
# os.chdir("/home/lw/clc_fairseq/TED-20210121-1335")

In [ ]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/opus-100-corpus.zip
!mkdir data_bin
!unzip 'opus-100-corpus.zip' -d 'data_bin'

In [ ]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

In [ ]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

In [ ]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item.strip() + "\n")
!ls -al

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-bert-large')

In [ ]:
start = time.time()
srcfile = "en_train"
tarfile = "vi_train"
lst_srcfile = en_train
lst_tarfile = vi_train

vec_lst_srcfile = model.encode(lst_srcfile)
print('=== Ending, Total time (second): ', time.time() - start)

# Calculate similarity

In [ ]:
import faiss
vec_lst_srcfile_copy = vec_lst_srcfile.copy()
start = time.time()
index = faiss.IndexFlatIP(vec_lst_srcfile_copy.shape[1])
index.ntotal
faiss.normalize_L2(vec_lst_srcfile_copy)
index.add(vec_lst_srcfile_copy)
k = 6
distance, index = index.search(vec_lst_srcfile_copy, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance as dt

count = 0
for i in range(len(lst_srcfile)):
    j = 1 #1 is index of biggest simmilarity
    score = distance[i][j]
    a = vec_lst_srcfile[i]
    b = vec_lst_srcfile[index[i][j]]
    
    dot_product = np.dot(a, b) # x.y
    norm_a = np.linalg.norm(a) #|x|
    norm_b = np.linalg.norm(b) #|y|
    sim_res = dot_product / (norm_a * norm_b)

    if score >=0.98 :
        count+=1
        print("==========================")
        for m in range(len(index[i])):
            print("--------", distance[i][m])
        print(i, index[i][j], distance[i][j], sim_res, 1-dt.cosine(a,b))
        print(lst_srcfile[i])
        print(lst_srcfile[index[i][j]])
        print("==========================")
print(count)

In [ ]:
start = time.time()
saperate = " || "

src_label = "S"
emb_label = "E"
thres_arr = [0.8, 0.85, 0.9, 0.95]
for i in range(len(thres_arr)):
    threshold = thres_arr[i]
    out_file_write = open("en_train_EM_"+str(thres_arr[i]), 'w')
    out_file_write_score = open("en_train_EM_score_"+str(thres_arr[i]), 'w')
    out_file_write_factor = open("en_train_EM_factor_"+str(thres_arr[i]), 'w')
    num_gth_thres = 0

    for i in range(len(lst_srcfile)):
        j=5
        score = distance[i][j]
        content = ""
        content_label_arr = []
        if score >= threshold and score <0.999:
            num_gth_thres += 1
            best_simi_index = index[i][j]
            content = lst_srcfile[i].strip() + saperate + lst_tarfile[best_simi_index].strip()

            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label.append(emb_label)
            content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())

            content_label_arr = " ".join(content_label)
        else:
            content = lst_srcfile[i].strip()
            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label_arr = " ".join(content_label)
 
        out_file_write.writelines(content+ "\n")    
        out_file_write_factor.writelines(content_label_arr + "\n")
        out_file_write_score.writelines(str(score) + "\n")

    print('threshold =', threshold, "EM sentences =", num_gth_thres, "/", len(lst_srcfile), num_gth_thres*100/len(lst_srcfile), "%" )

    out_file_write.close()
    out_file_write_score.close()
    out_file_write_factor.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

In [ ]:
print(len(lst_tarfile))